In [ ]:
from importlib import import_module
from pathlib import Path
import sys
sys.path.insert(0, "../")
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

# reload modules
import importlibs

import data
importlib.reload(data)

from data import read_data, read_labels, normalize_img

In [ ]:
# Read MNIST database
(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()

# Peprocessing (Normalization)
print('Raw data pixel value range:', train_data.min(), 'to', train_data.max())
train_data, train_labels = normalize_img(train_data, train_labels)
test_data, test_labels = normalize_img(test_data, test_labels)

print('Normalized datatye: ', type(train_data))
print('Normalized data pixel value range:', train_data.numpy().min(), 'to', train_data.numpy().max())

In [4]:
# Select model
model_type = 'cnn'  # Cambia a 'cnn' para usar la red convolucional

In [ ]:
# Define the path where the model is saved
OUTPUT_PATH = Path(f'./../../../saved_model/mnist_{model_type}') 

# Load the model
model = load_model(OUTPUT_PATH / 'model.h5')

model.summary()

In [7]:
# Get weights
# conv2d (3x3, 32) con padding='valid'
Wc0, bc0 = model.layers[0].get_weights()   # Wc0 shape [3,3,1,32] en Keras (KH,KW,Cin,Cout)
# Reordenar a [Cout, Cin, 3, 3] (si tu kernel lo espera así)
Wc0_o = np.transpose(Wc0, (3,2,0,1)).astype('float32').copy()
bc0_o = bc0.astype('float32').copy()
# Wc0_o.tofile(f'{OUTPUT_PATH}/conv0_W.bin'); bc0_o.tofile(f'{OUTPUT_PATH}/conv0_b.bin')

Wc1, bc1 = model.layers[3].get_weights()   # [3,3,32,64]
Wc1_o = np.transpose(Wc1, (3,2,0,1)).astype('float32').copy()
bc1_o = bc1.astype('float32').copy()
# Wc1_o.tofile(f'{OUTPUT_PATH}/conv1_W.bin'); bc1_o.tofile(f'{OUTPUT_PATH}/conv1_b.bin')

# Densas finales como antes:
W3,b3 = model.layers[7].get_weights()  # 1600->64
W4,b4 = model.layers[9].get_weights()  # 64->10


In [8]:

def save_dense(W, b, prefix, output_path):
    # Keras: W [in_dim, out_dim] → transponer a [out_dim, in_dim]
    W_rm = W.T.astype('float32').copy()
    b32  = b.astype('float32').copy()
    W_rm.tofile(f'{output_path}/{prefix}_W.bin')
    b32.tofile(f'{output_path}/{prefix}_b.bin')


In [ ]:
OUTPUT_PATH=Path(f'./../../opencl/weights/cnn_fp32')
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
Wc0_o.tofile(f'{OUTPUT_PATH}/conv0_W.bin'); bc0_o.tofile(f'{OUTPUT_PATH}/conv0_b.bin')
Wc1_o.tofile(f'{OUTPUT_PATH}/conv1_W.bin'); bc1_o.tofile(f'{OUTPUT_PATH}/conv1_b.bin')
save_dense(W3,b3,'fc0', OUTPUT_PATH)   # 128 x 784 en fichero
save_dense(W4,b4,'fc1', OUTPUT_PATH)   # 32  x 128